In [1]:
# !pip install beautifulsoup4
# !pip install requests

SyntaxError: invalid syntax (<ipython-input-1-894b866be2fa>, line 1)

In [1]:
from bs4 import BeautifulSoup as bs
import requests
import re
import csv

HEADERS = {
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64)...",
    "Accept-Language": "en-US, en;q=0.5",
}

In [ ]:
#get the list of best books of 2024
app_url = "https://www.goodreads.com"
start_url = "https://www.goodreads.com/choiceawards/best-books-2016"

res = requests.get(start_url, headers=HEADERS)
soup = bs(res.text, 'html.parser')

categories = soup.select('.category')

In [3]:
for index, category in enumerate(categories):
    genre = category.select('h4.category__copy')[0].text.strip()
    url = category.select('a')[0].get('href')
    category_url = f"{app_url}{url}"

In [4]:
res = requests.get(category_url, headers=HEADERS)
soup = bs(res.text, 'html.parser')

category_books = soup.select('.resultShown a.pollAnswer__bookLink')

In [5]:
def clean_string(string):
    cleaned = re.sub(r'\s+', ' ', string).strip()
    return cleaned

In [6]:
for book_index, book in enumerate(category_books):
    parent_tag = book.find_parent(class_='resultShown')
    votes = parent_tag.find(class_='result').text.strip()
    # print(votes)
    book_votes = clean_string(votes).split(" ")[0].replace(",", "")

In [7]:
#find information about each book
for book_index, book in enumerate(category_books):
  book_url = book.get('href')
  book_url_formatted = f"{app_url}{book_url}"
  book_img = book.find('img')
  book_img_url = book_img.get('src')
  book_img_alt = book_img.get('alt')
  book_title = clean_string(book_img_alt)
  print(book_title)
  book_name = book_title.split('by')[0].strip()
  book_author = book_title.split('by')[1].strip()

The Bookshop by Evan Friss
The Small and the Mighty by Sharon McMahon
The Demon of Unrest by Erik Larson
The Elements of Marie Curie by Dava Sobel
American Scary by Jeremy Dauber
Women in the Valley of the Kings by Kathleen Sheppard
Nexus by Yuval Noah Harari
Ask Not by Maureen Callahan
Challenger by Adam Higginbotham
The Lost Tomb by Douglas Preston
When Women Ran Fifth Avenue by Julie Satow
Not Your China Doll by Katie Gee Salisbury
Madness by Antonia Hylton
By the Fire We Carry by Rebecca Nagle
The Situation Room by George Stephanopoulos
The Wide Wide Sea by Hampton Sides
When the Sea Came Alive by Garrett M. Graff
The Siege by Ben Macintyre
Everyone Who Is Gone Is Here by Jonathan Blitzer
Eden Undone by Abbott Kahler


In [8]:
def get_ratings_reviews(text):

    print("text = ", text)
    # Find the substring for ratings
    ratings = text[:text.find(" ratings")].replace(",", "")

    # Find the substring for reviews
    reviews = text[text.find("and ") + 4:text.find(" reviews")].replace(",", "")

    return int(ratings), int(reviews)

In [9]:
res = requests.get(book_url_formatted, headers=HEADERS)
soup = bs(res.text, 'html.parser')

book_rating = soup.find(class_="RatingStatistics__rating").text.strip()
print(book_rating)

book_ratings_reviews = soup.find(class_="RatingStatistics__meta").get('aria-label').strip()
book_ratings, book_reviews = get_ratings_reviews(book_ratings_reviews)
print(f"Ratings: {book_ratings}, Reviews: {book_reviews}")

book_description_elements = soup.select('.BookPageMetadataSection__description .Formatted')
if book_description_elements:
    book_description = book_description_elements[0].text
else:
    book_description = 'No description found'
print(book_description)

3.76
text =  1,143 ratings and 213 reviews
Ratings: 1143, Reviews: 213
An incredible true story of murder in a utopian community established on a remote Galápagos island by European refugees and the American industrialist who became embroiled in the investigation—from the New York Times bestselling author of  The Ghosts of Eden ParkAt the height of the Great Depression, Los Angeles oil mogul George Allan Hancock and his crew of Smithsonian scientists came upon a gruesome scene: two bodies, mummified by the searing heat, on the shore of a remote Galápagos island. For the past four years Hancock and other American elites had traveled the South Seas to collect specimens for scientific research. On one trip to the Galápagos, Hancock was surprised to discover an equally exotic group of humans: European exiles who had fled political and economic unrest, hoping to create a utopian paradise. One was so devoted to a life of isolation that he’d had his teeth extracted and replaced with a set of 

In [10]:
print(soup)

<!DOCTYPE html>
<html lang="en"><head><meta charset="utf-8" data-next-head=""/><meta content="width=device-width" data-next-head="" name="viewport"/><link href="/favicon.ico" rel="icon"/><link href="https://www.goodreads.com/book/show/210246743-eden-undone" rel="canonical"/><link href="https://www.goodreads.com/en/book/show/210246743-eden-undone" hreflang="en" rel="alternate"/><link href="https://www.goodreads.com/nl/book/show/210246743-eden-undone" hreflang="nl" rel="alternate"/><meta content="Read 213 reviews from the world’s largest community 
    for readers. An incredible true story of murder in a utopian community established on a remote Galápago…" name="description"/><meta content="initial-scale=1.0, width=device-width" name="viewport"/><script type="application/ld+json">{"@context":"https://schema.org","@type":"Book","name":"Eden Undone: A True Story of Sex, Murder, and Utopia at the Dawn of World War II","image":"https://images-na.ssl-images-amazon.com/images/S/compressed.phot

In [11]:
#find number of people wanting to and currently reading
text = soup.get_text()

# Use regex to find the number before "people want to read"
match = re.search(r'([\d,]+)\s+people want to read', text)

if match:
    number_str = match.group(1).replace(",", "")
    want_to_read_num = int(number_str)
    print("Number of people who want to read:", want_to_read_num)

# Use regex to find the number before "people are currently reading"
match = re.search(r'([\d,]+)\s+people are currently reading', text)

if match:
    number_str = match.group(1).replace(",", "")
    current_read_num = int(number_str)
    print("Number of people who are people are currently reading:", current_read_num)


Number of people who want to read: 17943
Number of people who are people are currently reading: 306


In [16]:
# import json
# import re
# from bs4 import BeautifulSoup

# Step 1: Find all <script> tags
scripts = soup.find_all("script")

# Step 2: Search for BookGenre JSON inside those scripts
book_genres = []

for script in scripts:
    if script.string and 'BookGenre' in script.string:
        matches = re.findall(r'\{[^{}]*"__typename":"BookGenre"[^{}]*\}', script.string)
        for m in matches:
            try:
                data = json.loads(m + '}') if not m.endswith('}') else json.loads(m)
                name = data['genre']['name']
                url = data['genre']['webUrl']
                book_genres.append((name, url))
            except Exception as e:
                continue  # Skip malformed entries

# Step 3: Output
for name, url in book_genres:
    print(f"Genre: {name} | URL: {url}")

print(matches)


[]


In [18]:
def get_genres(soup):
    genres = []
    for a in soup.select('a[href*="/genres/"]'):
        genres.append(a.text)
    return genres

get_genres(soup)

['Nonfiction',
 'History',
 'True Crime',
 'Crime',
 'Audiobook',
 'Biography',
 'Mystery']

In [42]:
#add this histogram of book ratings
book_ratings_reviews_hist = soup.find_all(class_="RatingsHistogram__labelTotal")#.get('aria-label').strip()
hist = []
for i in book_ratings_reviews_hist:
  print(i.text.strip())
  hist.append(i.text.strip())

251 (21%)
468 (40%)
340 (29%)
72 (6%)
12 (1%)


In [49]:
#add the first page of book reviews
book_ratings_reviews_written = soup.find_all(class_="TruncatedContent__text TruncatedContent__text--large")#.get('aria-label').strip()
written_reviews = []
for i in book_ratings_reviews_written:
  written_reviews.append(i.text.strip())

An incredible true story of murder in a utopian community established on a remote Galápagos island by European refugees and the American industrialist who became embroiled in the investigation—from the New York Times bestselling author of  The Ghosts of Eden ParkAt the height of the Great Depression, Los Angeles oil mogul George Allan Hancock and his crew of Smithsonian scientists came upon a gruesome scene: two bodies, mummified by the searing heat, on the shore of a remote Galápagos island. For the past four years Hancock and other American elites had traveled the South Seas to collect specimens for scientific research. On one trip to the Galápagos, Hancock was surprised to discover an equally exotic group of humans: European exiles who had fled political and economic unrest, hoping to create a utopian paradise. One was so devoted to a life of isolation that he’d had his teeth extracted and replaced with a set of steel dentures.As Hancock and his fellow American explorers would witne

In [25]:
author_avatar_url_element = soup.select('.PageSection .AuthorPreview a.Avatar img.Avatar__image')
if author_avatar_url_element:
    author_avatar_url = author_avatar_url_element[0].get('src')
else:
    author_avatar_url = 'No Avatar found'

author_description_element = soup.select('.PageSection > .TruncatedContent .Formatted')
if author_description_element:
    author_description = author_description_element[0].text
else:
    author_description = 'No description found'

print(author_description)

bookPages = soup.select_one(".FeaturedDetails p[data-testid='pagesFormat']")
if bookPages:
    pages_format = bookPages.text[:bookPages.text.find(" pages")]
else:
    pages_format = "No pages found"
print(pages_format)

publication_info = soup.select_one(".FeaturedDetails p[data-testid='publicationInfo']")
if publication_info:
    publication = publication_info.text[16:]
else:
    publication = "No publication found"
print(publication)

This author is also published under  Karen AbbottAbbott Kahler, formerly writing as Karen Abbott, is the New York Times bestselling author of Sin in the Second City; American Rose; Liar, Temptress, Soldier, Spy; and The Ghosts of Eden Park. She is also the host of Remus: The Mad Bootleg King, a podcast about legendary Jazz Age bootlegger George Remus. A native of Philadelphia, she lives in New York City and in Greenport, New York.
352
September 24, 2024


In [62]:
book_dict = {
    "category": genre,
    "votes": book_votes,
    "title": book_name,
    "description": book_description,
    "author_name": book_author,
    "author_about": author_description,
    "avatar_url": author_avatar_url,
    "pages": pages_format,
    "rating": book_rating,
    "num_ratings": book_ratings,
    "num_reviews": book_reviews,
    "publication_info": publication,
    "img_url": book_img_url,
    "book_url": f"{app_url}{book_url}",
    "ratings_hist": hist,
    "written_reviews": written_reviews,
    "want_to_read_num": want_to_read_num,
    "current_read_num": current_read_num,
}

for i in book_dict:
  print(i, ":", book_dict[i])

print(book_dict.keys())

category : History & Biography
title : Eden Undone
description : An incredible true story of murder in a utopian community established on a remote Galápagos island by European refugees and the American industrialist who became embroiled in the investigation—from the New York Times bestselling author of  The Ghosts of Eden ParkAt the height of the Great Depression, Los Angeles oil mogul George Allan Hancock and his crew of Smithsonian scientists came upon a gruesome scene: two bodies, mummified by the searing heat, on the shore of a remote Galápagos island. For the past four years Hancock and other American elites had traveled the South Seas to collect specimens for scientific research. On one trip to the Galápagos, Hancock was surprised to discover an equally exotic group of humans: European exiles who had fled political and economic unrest, hoping to create a utopian paradise. One was so devoted to a life of isolation that he’d had his teeth extracted and replaced with a set of steel 

In [64]:
csv_filename = "books.csv"

if index == 0 and book_index == 0:
    with open(csv_filename, "w", newline="", encoding="utf-8") as csv_file:
        writer = csv.DictWriter(csv_file, fieldnames=book_dict.keys())
        writer.writeheader()


with open(csv_filename, mode="a", newline="", encoding="utf-8") as csv_file:
    writer = csv.DictWriter(csv_file, fieldnames=book_dict.keys())
    writer.writerow(book_dict)